# Housing Prices Prediction - Stacking,Feature Engineering & Outlier Handing

We would follow the below major steps:-

1. EDA

2. Handling missing  values

3. Feature Engineering

4. Outlier handling

5. Modelling

There are various steps other than the ones listed above which you would encounter in the notebook.

PS - Would suggest everyone to please check the Outlier Handling section as I haven't seen many notebooks using this method. Most of the notebooks have used manual methods. Cheers !!

Also please leave your suggestions to improve this notebook in the comments. Would be of great help in learning.

### Edit - For some interesting hyperparameter tuning techniques, check out the link below:-

https://www.kaggle.com/ankur123xyz/advanced-hyperparameter-tuning-techniques

Let us start of by importing all the necessary packages.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor , StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn.model_selection import KFold,cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

We import both the train and test sets into different dataframes

In [ ]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

We should check the dimensions of these dataframes before proceeding.
We have 1460 and 1459 rows in the train and test set respectively. We have 80 features in both the train and test set. The extra column in the train set would be the prediction.

In [ ]:
print("Dimensions of train set are:-",train.shape[0],",",train.shape[1])
print("Dimensions of test set are:-",test.shape[0],",",test.shape[1])

Let's expand our column view since we have 81 columns in the train set.

In [ ]:
pd.set_option("display.max_columns",81)

Having a peek at the train set we can see all features along with the value we have to predict - SalePrice

In [ ]:
train.head()

<a id="link1"></a>


## 1. Exploratory Data Analysis

To get a sense of which features are important and from where do we start we can create a correlation matrix. By default it uses the pearson correlation coefficent.


In [ ]:
corr = train.corr()

Having a look at the correlation of different numerical feature with SalePrice.

In [ ]:
corr["SalePrice"].sort_values(ascending=False)

Now we can zoom in to the 10 most important variables according to the pearson correlation coefficient and check the matrix.

In [ ]:
x=corr["SalePrice"].sort_values(ascending=False)[:11].index
mask = np.zeros_like(corr.loc[x,x])
mask[np.triu_indices_from(mask)]=True

sns.heatmap(corr.loc[x,x],mask=mask,annot=True,cmap="coolwarm")

Let us pair the 5 most important variables according to our matrix with sale price.\n
We can see some outliers, which we will take care of later.

The OverallQual and GarageCars plot with Sale Price will be better respresented with box plots, since they are ordinal features.


In [ ]:
sns.pairplot(train[x[0:6]],x_vars=list(x[1:6]),y_vars=[x[0]],diag_kind="kde")

OverallQual and GarageCards have a positive correlation with the Sale Price. Price for houses with 4 garages are lower which seems to be an anomaly or there might be some other feature which is impacting the . We have only 5 records with 4 Garage cars which can cause such cases.

In [ ]:
f,ax = plt.subplots(1,2,figsize=(15,8))
sns.boxplot("OverallQual","SalePrice",data=train,ax=ax[0])
sns.boxplot("GarageCars","SalePrice",data=train,ax=ax[1])

Now it is time to combine our train and test sets since we need to preprocess it the same way so that we can feed it later into our model.

Also we would drop the SalePrice column before merging and copy it to another series - y.

In [ ]:
index = len(train)
y=train["SalePrice"]
train.drop("SalePrice",axis=1,inplace=True)
dataset = pd.concat([train,test]).reset_index(drop=True)

<a id="link2"></a>


## 2. Handling missing values


Checking for null values in our new combined dataset.
    


In [ ]:
dataset.isnull().sum()[dataset.isnull().sum()>0].sort_values(ascending=False)

We impute the missing values for zone using neighborhood as an indicator.

In [ ]:
df_mszon=dataset.groupby(["Neighborhood","MSZoning"])["Id"].count().reset_index().groupby(["Neighborhood"])
maximum_mszon = df_mszon.max()
maximum_mszon= maximum_mszon.drop("Id",axis=1)

max_dict_mszon = maximum_mszon.to_dict()
def mapper_mszon(x):
    for index,val in zip(max_dict_mszon,max_dict_mszon.values()):
        for index1,val1 in val.items():
            if(x==index1):
                return val1

dataset.loc[dataset["MSZoning"].isnull(),"MSZoning"] = dataset.loc[dataset["MSZoning"].isnull(),"Neighborhood"].apply(lambda x: mapper_mszon(x))

Similarly we impute the values for LotFrontage using Neighborhood and LotConfig as indicators. We taken the median of similar Neighborhood and LotFrontage values.

In [ ]:
df_lotf=dataset.groupby(["Neighborhood","LotConfig"])["LotFrontage"].mean()

max_dict_lotf = df_lotf.to_dict()
def mapper_lotf(x1,x2):
    for index,val in zip(max_dict_lotf,max_dict_lotf.values()):
        if((x1==index[0]) & (x2==index[1])):
            return val

dataset.loc[dataset["LotFrontage"].isnull(),"LotFrontage"] = dataset.loc[dataset["LotFrontage"].isnull(),["Neighborhood","LotConfig"]].apply(lambda x: mapper_lotf(x[0],x[1]),axis=1)
dataset["LotFrontage"] = dataset.groupby(["Neighborhood"])["LotFrontage"].transform(lambda x: x.median())

Base on the definition of these features give in the data description we impute the rest of the values to NA or 0 which signifies absence of that feature.

In [ ]:
dataset["Alley"].fillna("NA",inplace=True)
dataset["Utilities"].fillna("NA",inplace=True)
dataset["Exterior1st"].fillna("NA",inplace=True)
dataset["Exterior2nd"].fillna("NA",inplace=True)
dataset["MasVnrType"].fillna("NA",inplace=True)
dataset["MasVnrArea"].fillna(0,inplace=True)
dataset["BsmtQual"].fillna("NA",inplace=True)
dataset["BsmtCond"].fillna("NA",inplace=True)
dataset["BsmtExposure"].fillna("No",inplace=True)
dataset["BsmtFinType1"].fillna("NA",inplace=True)
dataset["BsmtFinSF1"].fillna(0,inplace=True)
dataset["BsmtFinType2"].fillna("NA",inplace=True)
dataset["BsmtFinSF2"].fillna(0,inplace=True)
dataset["BsmtUnfSF"].fillna(0,inplace=True)
dataset["TotalBsmtSF"].fillna(0,inplace=True)
dataset["BsmtFullBath"].fillna(0,inplace=True)
dataset["BsmtHalfBath"].fillna(0,inplace=True)
dataset["FireplaceQu"].fillna("NA",inplace=True)
dataset["GarageType"].fillna("NA",inplace=True)
dataset["GarageYrBlt"].fillna(0,inplace=True)
dataset["GarageFinish"].fillna("NA",inplace=True)
dataset["GarageCars"].fillna(0,inplace=True)
dataset["GarageArea"].fillna(0,inplace=True)
dataset["GarageQual"].fillna("NA",inplace=True)
dataset["GarageCond"].fillna("NA",inplace=True)
dataset["PoolQC"].fillna("NA",inplace=True)
dataset["Fence"].fillna("NA",inplace=True)
dataset["MiscFeature"].fillna("NA",inplace=True)

For features that can't be missing we have taken the mode and the default value as per data description.

In [ ]:
dataset["Functional"].fillna("Typ",inplace=True)
dataset["Electrical"].fillna(dataset["Electrical"].mode()[0],inplace=True)
dataset["KitchenQual"].fillna(dataset["KitchenQual"].mode()[0],inplace=True)
dataset["SaleType"].fillna("Oth",inplace=True)

Since the SubClass are categories and not of numeric data type we covert the feature to category type.

In [ ]:
dataset["MSSubClass"]=dataset["MSSubClass"].astype("category")

Let us check the distribution of our dependent variable. It doesnt seem to be a normal distribution and leans towards a positive skewed distribution.

In [ ]:
x=sns.distplot(y)
x.set_title("Distribution plot for Sale Price")

Let us log transform it and recheck. Post log transform it is close to a normal distribution.

In [ ]:
x=sns.distplot(np.log1p(y),fit=norm)
x.set_title("Distribution plot for Sale Price")

We log transform SalePrice as we have seen earlier it helps in getting a normal distribution.

In [ ]:
y= np.log1p(y)

<a id="link3"></a>



## 3. Feature Engineering

We have seen quite a few of the non numerical features have ordinal nature. We have transformed them below to numeric type to maintain the ordinal relation.
    
    


In [ ]:
dataset = dataset.replace({"Alley" : {"Grvl" : 1, "Pave" : 2,"NA":0},
                       "BsmtCond" : {"NA" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtQual" : {"NA" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"NA" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"NA" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"NA" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageFinish" : {"Fin" : 3, "RFn" : 2, "Unf" : 1, "NA" : 0},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"NA" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2, "NA":0},
                       "Utilities" : {"ELO" : 1, "NASeWa" : 2, "NASewr" : 3, "AllPub" : 4}}
                     )

We have created flags for different feature of the house to check if it is available or not.

In [ ]:
cats = ["basement_flag","fire_flag","wooddeck_flag","garage_flag","pool_flag","fence_flag"]
dataset["basement_flag"]=np.where(dataset["TotalBsmtSF"]>0,1,0)
dataset["fire_flag"]=np.where(dataset["Fireplaces"]>0,1,0)
dataset["wooddeck_flag"]=np.where(dataset["WoodDeckSF"]>0,1,0)
dataset["garage_flag"]=np.where(dataset["GarageArea"]>0,1,0)
dataset["porch_flag"]=np.where((dataset["OpenPorchSF"]+dataset["EnclosedPorch"]+dataset["3SsnPorch"]\
                                +dataset["ScreenPorch"])>0,1,0)
dataset["pool_flag"] = np.where(dataset["PoolArea"]>0,1,0)
dataset["fence_flag"] = np.where(dataset["Fence"]=="NA",1,0)
dataset[cats]=dataset[cats].astype("category")

We create a TotalSF feature which includes the surface area for basement, 1st floor and the 2nd floor.

In [ ]:
dataset['TotalSF'] = dataset['TotalBsmtSF'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']

<a id="link4"></a>



## 4. Outlier Handling 
Outlier handling is done using the mean and the standard deviation. We have taken three features which have highest correlation to the sale price based on which outliers have been removed.
    


In [ ]:
outlier_features = ["GrLivArea","TotalSF","GarageArea"]

means,sds = np.mean(dataset[outlier_features]),np.std(dataset[outlier_features])
lower,upper = means - 2.5*sds , means + 2.5*sds
def compare(x):
    count=0
    for col in outlier_features:
        if x[col]>lower[col] and x[col]<upper[col]:
            count=count+1
    if count==len(outlier_features):
        return True
    else: 
        return False

y= y.array
sub_data = dataset.loc[:index-1]  
test_1 = dataset.loc[index:,:]

data_train_new = sub_data.loc[sub_data.apply(lambda x: compare(x),axis=1)]
y= y[sub_data.apply(lambda x: compare(x),axis=1)]

index = len(data_train_new)
dataset = pd.concat([data_train_new,test_1]).reset_index(drop=True)

For all the numeric features we have checked for skew and have log transformed those features which have high skew (greater than 0.5 in our case) to get a normal distribtuion.

In [ ]:
numeric_feats = dataset.dtypes[(dataset.dtypes != "object") & (dataset.dtypes != "category")].index
skewed_feats = dataset[numeric_feats].apply(lambda x: skew(x)).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness[abs(skewness) > 0.5]

skewed_features = skewness.index
dataset[skewed_features] = np.log1p(dataset[skewed_features])

We are dropping the Id feature since it would not add any useful information to the model.

In [ ]:
dataset.drop("Id",axis=1,inplace=True)

We one hot encode the dataset and then split it to the orginal train set (with outliers removed) and the test set(all records intact).

In [ ]:
dataset = pd.get_dummies(dataset, drop_first= True)
train_1 = dataset.loc[:index-1,:]
test_1 = dataset.loc[index:,:]

Let us now split out train set further in to train and test sets for validation.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_1,y,test_size=0.2, random_state=42)

Since we have a high number of variables and is prone to have outliers as we worked only on a few features we should opt for Robust Scaler transformation to handle the outliers.
Below is a good read on different scaling methods:-
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

In [ ]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
X_train= rs.fit_transform(X_train)
X_test = rs.transform(X_test)
X_submit = rs.transform(test_1.values)

<a id="link5"></a>



## 5. Modelling

We use 10 fold KFold for cross validation.

We have used the following models:-
1. Random Forest
2. XGBoost
3. Gradient Boosting
4. Support Vector
5. Ridge (L2)
6. Lasso (L1)

We have run these models individually using a grid search for the right set of parameters.
Then we have used the best models in all of them and used them in our stacking model.
    


In [ ]:
kf = KFold(n_splits=10,shuffle=True,random_state=42)
rf = RandomForestRegressor(n_estimators=500)
xgb = XGBRegressor()
gb= GradientBoostingRegressor()
svr =SVR(C=70,epsilon=.115)
ridge = Ridge(alpha=1.75,solver ="lsqr")
lasso = Lasso(alpha=.0009)

clf=[\
     ("XGBoost",xgb,{"n_estimators":[200],"max_depth":[3],"learning_rate":[.1],"subsample":[1],"colsample_bytree":[1],"gamma":[0],"lambda":[.001,.01]})\
     ,("Gradient Boost",gb,{})\
     ,("SVR",svr,{"C":[60,65,70,75,80],"epsilon":[.11,.115,.12]})\
    ,("Ridge",ridge,{"alpha":[1.4,1.5,1.6,1.75,1.8]})\
     ,("Lasso",lasso,{"alpha":[.0005,.0006,.0007,.001,.01,.1]})]

est=[]
results_data=pd.DataFrame(columns=["Name","Train_Score","Test_Score"])

i=0
for name,reg,param_grid in clf:
    gs= GridSearchCV(reg,param_grid=param_grid,cv=kf,scoring="neg_mean_squared_error")
    gs.fit(X_train,y_train)
    y_pred = gs.best_estimator_.predict(X_test)
    test_score = np.sqrt(mean_squared_error(y_test, y_pred))
    results_data.loc[i,]= [name,-gs.best_score_,test_score]
    i=i+1
    est.append([name,gs.best_estimator_])
    
sc = StackingRegressor(estimators=est[1:],final_estimator= ridge,cv=kf)
sc.fit(X_train,y_train)
y_trn= sc.predict(X_train)
y_tst= sc.predict(X_test)    
train_score = np.sqrt(mean_squared_error(y_train, y_trn))
test_score = np.sqrt(mean_squared_error(y_test, y_tst))
results_data.loc[i,]= ["Stack",train_score,test_score]

Checking the train and test results for all the models.

In [ ]:
results_data

Since stacking gave us the best scores we would used that to get the predictions to be used to submit our scores.
We would do an inverse log transform since we had log transformed the SalePrice earlier.

In [ ]:
y_submit= np.expm1(sc.predict(X_submit))

Creating a dataset which would be submitted for evaluation.

In [ ]:
data_submit= pd.DataFrame({"Id":test.Id,"SalePrice":y_submit})

Exporting our submission dataset to a csv file.

In [ ]:
data_submit.to_csv("submit_housing.csv",index=False)